In [252]:
import numpy as np
import pandas as pd
from io import StringIO

In [253]:
DATA_FOLDER = './data/'
TRAIN_FILE = './train.csv'
TEST_FILE = './train.csv'

CUT_TRAIN_FILE = './train_cut.csv'

In [254]:
# to split the initial file
# pd_df = pd.read_csv(DATA_FOLDER + TRAIN_FILE, sep=',', )
# pd_df.iloc[:50].to_csv(DATA_FOLDER + 'train_cut.csv', sep=',', index=False)

In [255]:
# import pandas as pd
pd_df = pd.read_csv(DATA_FOLDER + CUT_TRAIN_FILE, sep=',', )
pd_df

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,100000,s,138.470,51.655,97.827,27.980,0.910,124.711,2.666,3.064,...,-0.277,258.733,2,67.435,2.150,0.444,46.062,1.240,-2.475,113.497
1,100001,b,160.937,68.768,103.235,48.146,-999.000,-999.000,-999.000,3.473,...,-1.916,164.546,1,46.226,0.725,1.158,-999.000,-999.000,-999.000,46.226
2,100002,b,-999.000,162.172,125.953,35.635,-999.000,-999.000,-999.000,3.148,...,-2.186,260.414,1,44.251,2.053,-2.028,-999.000,-999.000,-999.000,44.251
3,100003,b,143.905,81.417,80.943,0.414,-999.000,-999.000,-999.000,3.310,...,0.060,86.062,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
4,100004,b,175.864,16.915,134.805,16.405,-999.000,-999.000,-999.000,3.891,...,-0.871,53.131,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
5,100005,b,89.744,13.550,59.149,116.344,2.636,284.584,-0.540,1.362,...,2.237,282.849,3,90.547,-2.412,-0.653,56.165,0.224,3.106,193.660
6,100006,s,148.754,28.862,107.782,106.130,0.733,158.359,0.113,2.941,...,-1.443,294.074,2,123.010,0.864,1.450,56.867,0.131,-2.767,179.877
7,100007,s,154.916,10.418,94.714,29.169,-999.000,-999.000,-999.000,2.897,...,-1.761,187.299,1,30.638,-0.715,-1.724,-999.000,-999.000,-999.000,30.638
8,100008,b,105.594,50.559,100.989,4.288,-999.000,-999.000,-999.000,2.904,...,0.024,129.804,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
9,100009,s,128.053,88.941,69.272,193.392,-999.000,-999.000,-999.000,1.609,...,0.845,294.741,1,167.735,-2.767,-2.514,-999.000,-999.000,-999.000,167.735


In [256]:
def load_csv(path):
    csv = np.genfromtxt(DATA_FOLDER + path, delimiter=',', skip_header=0, names = True, dtype = None)
    for elt in csv:
        elt[1] = 0 if(elt[1]==bytes('b',"ascii")) else 1
    return csv

train_df = load_csv(CUT_TRAIN_FILE)

train_df


<ipython-input-256-1f9b10f8fc29>:2: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  csv = np.genfromtxt(DATA_FOLDER + path, delimiter=',', skip_header=0, names = True, dtype = None)


array([(100000, b'1',  138.47 ,  51.655,  97.827,  27.98 ,  9.100e-01,  124.711,  2.666e+00, 3.064, 41.928, 197.76 , 1.582,  1.396,  2.00e-01,  32.638,  1.017,  0.381,  51.626,  2.273, -2.414,  16.824, -0.277, 258.733, 2,   67.435,  2.150e+00,  4.440e-01,   46.062,  1.240e+00, -2.475e+00, 113.497),
       (100001, b'0',  160.937,  68.768, 103.235,  48.146, -9.990e+02, -999.   , -9.990e+02, 3.473,  2.078, 125.157, 0.879,  1.414, -9.99e+02,  42.014,  2.039, -3.011,  36.918,  0.501,  0.103,  44.704, -1.916, 164.546, 1,   46.226,  7.250e-01,  1.158e+00, -999.   , -9.990e+02, -9.990e+02,  46.226),
       (100002, b'0', -999.   , 162.172, 125.953,  35.635, -9.990e+02, -999.   , -9.990e+02, 3.148,  9.336, 197.814, 3.776,  1.414, -9.99e+02,  32.154, -0.705, -2.093, 121.409, -0.953,  1.052,  54.283, -2.186, 260.414, 1,   44.251,  2.053e+00, -2.028e+00, -999.   , -9.990e+02, -9.990e+02,  44.251),
       (100003, b'0',  143.905,  81.417,  80.943,   0.414, -9.990e+02, -999.   , -9.990e+02, 3.31 , 